# Production route for products route

TLDR;  
to bypass an authed route, post 
- with authentication header 'Authentication: Bearer token'
- token got from a post to <host>/api/auth/token/ with the json payload {"username": "emailaddress", "password": "secret"}

In [6]:
import httpx

In [7]:
server = "http://localhost:8000"
email = "user@example.com"
password = "secret"

In [8]:
# Is the server alive?
url = f"{server}/api/"
r = httpx.get(url)
assert r.json()['status'] == "up and running"

In [9]:
# Get the token
url = f"{server}/api/auth/token/"
r = httpx.post(url, json={"email": email, "password": password})

In [12]:
# The authentication token is provided in the cookies.
# The Refresh token is in the reponse-body.
refresh_token = r.json()["refresh"]
authentication_cookies = {"Authentication": r.cookies['Authentication'].strip('"')}

In [13]:
r = httpx.post(f"{server}/api/auth/token/verify/", cookies=authentication_cookies)
assert r.status_code == 200
assert r.json() == {'detail': 'Token is valid.'}

In [14]:
refresh_url = f"{server}/api/auth/token/refresh/"

payload = {"refresh": refresh_token}
r = httpx.post(refresh_url, json=payload, cookies=authentication_cookies)

assert r.status_code == 200
assert "refresh" in r.json()

# Save new tokens
refresh_token = r.json()["refresh"]
authentication_cookies = {"Authentication": r.cookies['Authentication'].strip('"')}

In [15]:
logout_url = f"{server}/api/auth/token/logout/"
payload = {"refresh": refresh_token}

r = httpx.post(logout_url, json=payload, cookies=authentication_cookies)

assert r.status_code == 200
assert r.json() == {'detail': 'Successfully logged out.', 'redirect': '/api/'}